In [1]:
import numpy as np
import xarray as xr
import warnings
import fsspec
warnings.filterwarnings("ignore")

In [2]:
run token.ipynb

Stored 'token' (dict)


### Daily precip, smb, snowfall, subl, t2m, and tskin data

daily data from 1979-2018

#### Reading in files

In [3]:
data_loc = '/home/jovyan/data/RACMO/daily/'

var_names = np.array(['precip','smb','snowfall','snowmelt','subl','t2m','tskin'])
year_strings = np.array(['1979','1981','1991','2001','2011'])

In [4]:
racmo_dict={}
for var in var_names:
    filename_array = np.array([])
    for year in year_strings:
        fname = data_loc + var + '.KNMI-' + year + '.XPEN055.ERAINx_RACMO2.3p2.DD.nc'
        filename_array = np.append(filename_array,fname)
    racmo_var = xr.open_mfdataset(filename_array,combine='nested',compat='override')
    racmo_dict[var] = racmo_var

In [5]:
racmo_1 = xr.merge([racmo_dict[key] for key in racmo_dict],compat='override')

#### Splitting up chunks

In [6]:
da_names = list(racmo_1.data_vars.keys())
new_chunk_sizes = {
    'dir':{'rlat':100,'rlon':100},
    'time_bnds':{'time':2000},
    'dtg':{'time':2000},
    'date_bnds':{'time':2000},
    'hms_bnds':{'time':2000},
    'assigned':{'time':2000},
    'rotated_pole':{},
    'precip':{'time':2000,'rlat':100,'rlon':100},
    'smb':{'time':2000,'rlat':100,'rlon':100},
    'snowfall':{'time':2000,'rlat':100,'rlon':100},
    'snowmelt':{'time':2000,'rlat':100,'rlon':100},
    'subl':{'time':2000,'rlat':100,'rlon':100},
    't2m':{'time':2000,'rlat':100,'rlon':100},
    'tskin':{'time':2000,'rlat':100,'rlon':100},
}

In [7]:
racmo = xr.Dataset()
for da_name in da_names:
    da = racmo_1[da_name]
    da_chunked = da.chunk(new_chunk_sizes.get(da_name,{}))
    racmo[da_name] = da_chunked

In [8]:
racmo.t2m.chunks

((2000, 2000, 2000, 2000, 2000, 2000, 2000, 853),
 (1,),
 (100, 100, 100, 100, 54),
 (100, 100, 100, 100, 100, 1))

In [9]:
racmo.nbytes/1e9

94.60101706

#### Writing to Google Bucket

In [ ]:
filename = 'gs://ldeo-glaciology/RACMO/AP_new'
mapper = fsspec.get_mapper(filename, mode='w',token=token)
racmo.to_zarr(mapper)

### 3-hourly t2m data

For 2m atmospheric temperature, we have 3-hour resolution data  from 2016-2021. This is loaded into a separate dataset below.

#### Reading in files

In [10]:
data_loc = '/home/jovyan/data/RACMO/t2m/'

year_strings = np.array(['2016','2021'])

In [11]:
racmo_3H_dict={}
filename_array = np.array([])
for year in year_strings:
    fname = data_loc + 't2m.KNMI-' + year + '.XPEN055.ERA5-3H_RACMO2.3p2.3H.nc'
    filename_array = np.append(filename_array,fname)
racmo_3H_1 = xr.open_mfdataset(filename_array,combine='nested',compat='override')

#### Splitting up chunks

In [12]:
da_names = list(racmo_3H_1.data_vars.keys())
new_chunk_sizes = {
    'dir':{'rlat':100,'rlon':100},
    'time_bnds':{'time':2000},
    'dtg':{'time':2000},
    'date_bnds':{'time':2000},
    'hms_bnds':{'time':2000},
    'assigned':{'time':2000},
    'rotated_pole':{},
    'precip':{'time':2000,'rlat':100,'rlon':100},
    'smb':{'time':2000,'rlat':100,'rlon':100},
    'snowfall':{'time':2000,'rlat':100,'rlon':100},
    'snowmelt':{'time':2000,'rlat':100,'rlon':100},
    'subl':{'time':2000,'rlat':100,'rlon':100},
    't2m':{'time':2000,'rlat':100,'rlon':100},
    'tskin':{'time':2000,'rlat':100,'rlon':100},
}

In [13]:
racmo_3H = xr.Dataset()
for da_name in da_names:
    da = racmo_3H_1[da_name]
    da_chunked = da.chunk(new_chunk_sizes.get(da_name,{}))
    racmo_3H[da_name] = da_chunked

In [14]:
racmo_3H.t2m.chunks

((2000, 2000, 2000, 2000, 2000, 2000, 2000, 1088),
 (1,),
 (100, 100, 94),
 (100, 100, 100, 90))

In [15]:
racmo_3H.nbytes/1e9

6.92380574

#### Writing to Google Bucket

In [ ]:
filename = 'gs://ldeo-glaciology/RACMO/AP_3H'
mapper = fsspec.get_mapper(filename, mode='w',token=token)
racmo.to_zarr(mapper)